In [ ]:
import pandas as pd
import transformers
import torch
import random
import pandas as pd
import random
import string
import numpy as np
import unidecode
from unidecode import unidecode
from wordcloud import STOPWORDS
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from torch.utils.data import TensorDataset
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer, BertModel, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, PreTrainedTokenizerFast
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn import metrics
import tensorflow as tf
from tqdm.notebook import tqdm
import numpy as np
import torch
transformers.logging.set_verbosity_debug()
from loguru import logger
logger.add("logging/finnlp_bert_experiment_1.log", rotation="500 MB")

In [5]:
BATCH_SIZE = 31

In [6]:
logger.debug("<<<<<<<<<< INICIOU >>>>>>>>>>>")
brc = pd.read_csv("Banking_Regulation_Corpora_BRC_anonymous.csv" ,encoding="utf-8", low_memory=False, sep=";")

2024-03-01 19:09:27.516 | DEBUG    | __main__:<module>:1 - <<<<<<<<<< INICIOU >>>>>>>>>>>


In [8]:
brc.head(2)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
0,1,DPT_25,2020-12-23,2128,36560,2020-12-23,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,656,67.0,127.0,COMUNICADO,771984,https://www.bcb.gov.br/estabilidadefinanceira/...
1,1,DPT_25,2020-12-28,2129,36569,2020-12-28,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,2110,156.0,322.0,COMUNICADO,772973,https://www.bcb.gov.br/estabilidadefinanceira/...


In [9]:
brc.keys()

Index(['class', 'department', 'entry_date', 'general_id',
       'normative_identifier', 'publication_date', 'regulatory_authority',
       'subject', 'subject_length', 'subject_unique_words', 'subject_words',
       'text', 'text_length', 'text_unique_words', 'text_words', 'type',
       'unique_document_id', 'url'],
      dtype='object')

In [10]:
brc.count()

class                   61864
department              61864
entry_date              61864
general_id              61864
normative_identifier    61864
publication_date        61864
regulatory_authority    61864
subject                 61864
subject_length          61864
subject_unique_words    61864
subject_words           61864
text                    61864
text_length             61864
text_unique_words       61864
text_words              61864
type                    61864
unique_document_id      61864
url                     61864
dtype: int64

In [11]:
brc = brc[brc.text_words >= 50]
brc.count()

class                   60836
department              60836
entry_date              60836
general_id              60836
normative_identifier    60836
publication_date        60836
regulatory_authority    60836
subject                 60836
subject_length          60836
subject_unique_words    60836
subject_words           60836
text                    60836
text_length             60836
text_unique_words       60836
text_words              60836
type                    60836
unique_document_id      60836
url                     60836
dtype: int64

In [12]:
sel_cols = [column for column, is_type in (brc.dtypes=="object").items() if is_type]
sel_cols

['department',
 'entry_date',
 'normative_identifier',
 'publication_date',
 'regulatory_authority',
 'subject',
 'text',
 'type',
 'url']

In [13]:
brc['class'] = brc['class'].astype(str)

In [14]:
brc = brc.rename(columns={"class":"label"})

In [16]:
brc.count()

label                   60836
department              60836
entry_date              60836
general_id              60836
normative_identifier    60836
publication_date        60836
regulatory_authority    60836
subject                 60836
subject_length          60836
subject_unique_words    60836
subject_words           60836
text                    60836
text_length             60836
text_unique_words       60836
text_words              60836
type                    60836
unique_document_id      60836
url                     60836
dtype: int64

In [17]:
class Model_BERT():
    
    def model_path_def(self, name_model):
        if name_model == "BERTIMBAU_BASE":
            model_path = MODEL_PATH
        elif name_model == "XXXX":
            model_path = MODEL_PATH_NEW
        else:
            print("Nome de modelo inválido")
            model_path = "None"
            
        return model_path

    def tokenizer_model(self, model_path, df, token):
        tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

        # Tokenizing the training data
        encoded_data_train = tokenizer.batch_encode_plus(
            df[df.data_type=='train'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True, 
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )

        # Tokenizing test data
        encoded_data_val = tokenizer.batch_encode_plus(
            df[df.data_type=='val'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True,
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )


        input_ids_train = encoded_data_train['input_ids']
        attention_masks_train = encoded_data_train['attention_mask']
        labels_train = torch.tensor(df[df.data_type=='train'].label_num.values)

        input_ids_val = encoded_data_val['input_ids']
        attention_masks_val = encoded_data_val['attention_mask']
        labels_val = torch.tensor(df[df.data_type=='val'].label_num.values)

        dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
        dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
        
        return(dataset_train, dataset_val)

    def setup_model(self, model_path, dataset_train, dataset_val, epochs):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        batch_size = BATCH_SIZE
        dataloader_train = DataLoader(dataset_train, 
                                      sampler=RandomSampler(dataset_train), 
                                      batch_size=batch_size)
        dataloader_validation = DataLoader(dataset_val, 
                                           sampler=SequentialSampler(dataset_val), 
                                           batch_size=batch_size)
        optimizer = AdamW(model.parameters(),
                          lr=1e-5, 
                          eps=1e-8)

        scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps=100,
                                                    num_training_steps=len(dataloader_train)*epochs)
        return(model, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device)


    def f1_score_func(self, preds, labels):
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        scr = f1_score(labels_flat, preds_flat, average='binary')
        logger.info(f"f1_score_binary: {scr}")
        return scr

    def accuracy_per_class(self, preds, labels):
        label_dict_inverse = {v: k for k, v in LABEL_DICT.items()}

        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        for label in np.unique(labels_flat):
            y_preds = preds_flat[labels_flat==label]
            y_true = labels_flat[labels_flat==label]
            print(f'Class: {label_dict_inverse[label]}')
            print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            print(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
            logger.info(f'Class: {label_dict_inverse[label]}')
            logger.info(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            logger.info(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
        
    def evaluate(self, model, dataloader_val, device):

        model.eval()

        loss_val_total = 0
        predictions, true_vals = [], []

        for batch in dataloader_val:

            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            with torch.no_grad():        
                outputs = model(**inputs)

            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)

        loss_val_avg = loss_val_total/len(dataloader_val) 

        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)

        logger.info(f"loss_val_avg, predictions, true_vals: {loss_val_avg}, {predictions}, {true_vals}")

        return loss_val_avg, predictions, true_vals

    def results(self, model_path, model_name, epochs, dataloader_validation, device, token):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        model.to(device)

        model.load_state_dict(torch.load(f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epochs}.model', map_location=torch.device('cpu')))

        _, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
        Model_BERT().accuracy_per_class(predictions, true_vals)
        from sklearn import metrics
        preds_flat = np.argmax(predictions, axis=1).flatten()
        metrics_report = metrics.classification_report(true_vals,preds_flat, output_dict=True)
        logger.info(metrics_report)
        
        return metrics_report
    

    def training_model(self, model, model_name, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device, token):
        seed_val = 1
        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        epoch_array = []
        f1_array = []
        train_loss_array = []
        val_loss_array = []

        for epoch in tqdm(range(1, epochs+1)):

            model.train()

            loss_train_total = 0

            progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
            for batch in progress_bar:

                model.zero_grad()

                batch = tuple(b.to(device) for b in batch)

                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'labels':         batch[2],
                         }       

                outputs = model(**inputs)

                loss = outputs[0]
                loss_train_total += loss.item()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                optimizer.step()
                scheduler.step()

                progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


            torch.save(model.state_dict(),f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}.model')
            model.save_pretrained(f"{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}")
            tqdm.write(f'\nEpoch {epoch}')
            print(f'\nEpoch {epoch}')
            logger.info(f'\nEpoch {epoch}')

            loss_train_avg = loss_train_total/len(dataloader_train)            
            tqdm.write(f'Training loss: {loss_train_avg}')
            logger.info(f'Training loss: {loss_train_avg}')

            val_loss, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
            val_f1 = Model_BERT().f1_score_func(predictions, true_vals)
            tqdm.write(f'Validation loss: {val_loss}')
            print(f'Validation loss: {val_loss}')
            logger.info(f'Validation loss: {val_loss}')
            
            tqdm.write(f'F1 Score (Weighted/Binary): {val_f1}')
            print(f'F1 Score (Weighted/Binary): {val_f1}')
            logger.info(f'F1 Score (Weighted/Binary): {val_f1}')
            
            epoch_array.append(epoch)
            f1_array.append(val_f1)
            val_loss_array.append(val_loss)
            train_loss_array.append(loss_train_avg)

        return(epoch_array, f1_array, val_loss_array, train_loss_array)
    
    def read_df(self, df):
        df['label_num'] = df.label.replace(LABEL_DICT)

        X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label_num.values, 
                                                  test_size=0.4, 
                                                  random_state=42, 
                                                  stratify=df.label.values)
        df['data_type'] = ['not_set']*df.shape[0]
        df.loc[X_train, 'data_type'] = 'train'
        df.loc[X_val, 'data_type'] = 'val'
        
        return(df)

    def call_setup(self, model_name, df, token, epochs):
        df = Model_BERT().read_df(df)
        model_path = Model_BERT().model_path_def(model_name)
        print(model_path)
        dataset_train, dataset_val = Model_BERT().tokenizer_model(model_path, df, token)
        model, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device = Model_BERT().setup_model(model_path, dataset_train, dataset_val, epochs)
        epoch_array, f1_array, val_loss_array, train_loss_array = Model_BERT().training_model(model, model_name, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device, token)
        df_performa = pd.DataFrame()
        df_performa["epoch"] = epoch_array
        df_performa["f1"] = f1_array
        df_performa["val_loss"] = val_loss_array
        df_performa["train_loss"] = train_loss_array
        metrics_report = Model_BERT().results(model_path,model_name,epochs,dataloader_val, device, token)
        logger.info(f'Metric Report Call Setup: {metrics_report}')
        
        return metrics_report, df_performa

In [18]:
departments = ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']

In [19]:
brc.keys()

Index(['label', 'department', 'entry_date', 'general_id',
       'normative_identifier', 'publication_date', 'regulatory_authority',
       'subject', 'subject_length', 'subject_unique_words', 'subject_words',
       'text', 'text_length', 'text_unique_words', 'text_words', 'type',
       'unique_document_id', 'url'],
      dtype='object')

In [20]:
def remove_duplicates(sentence):
	words = sentence.split(" ")
	result = []
	for word in words:
		if word not in result:
			result.append(word)
	return result

In [21]:
def cleaning(df,column):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    return df

In [ ]:
for department in departments:
    logger.debug(f"Department: {department}")
    df_modelagem = brc[brc.department == department]

    df_modelagem = df_modelagem.dropna()
    df_modelagem = df_modelagem.reset_index(drop=True)
    df_modelagem = df_modelagem.rename(columns={"text":"content"})

    df_modelagem = cleaning(df_modelagem,"content")

    new_content = []
    for texto in df_modelagem.content:
        new_content.append(texto.replace("[BUSINESS NEED]", ""))
    df_modelagem["new_content"] = new_content

    # Converting the text column into one long string.
    logger.debug("[+] Start converting the text to one string")
    stringssss = df_modelagem.to_string(columns=["content"], header=False, index=False, index_names=False)
    logger.debug("[+] End converting the text to one string")

    logger.debug("[+] Start removing duplicated words from the long string")
    unique_words_dts = remove_duplicates(stringssss)
    logger.info(f"unique_words_dts: {len(unique_words_dts)} \n")
    logger.debug("[+] End removing duplicated words from the long string")
    
    LABEL_DICT = {'0': 0, '1': 1}

    MODEL_SAVE_PATH = "notebooks/testes/scoring"
    BERTIMBAU_BASE = "datasets/BERTIMBAU_BASE"
    BERTIMBAU_TOKENIZER = "tokenizer/BERTimbau_Tokenizer"

    MODEL_PATH = BERTIMBAU_BASE
    MODEL_PATH_NEW = BERTIMBAU_BASE

    tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

    logger.debug(f"Department count samples: {df_modelagem.count()}")
    logger.debug(f"Department count samples class 1: {df_modelagem[df_modelagem.label == '1'].count()}")
    logger.debug(f"Department count samples class 0: {df_modelagem[df_modelagem.label == '0'].count()}")
    

    Model_BERT().call_setup("BERTIMBAU_BASE", df_modelagem, 512, 5)

<h2>The output result is in the next cell. It was copied from the log file.</h2>

In [ ]:
# 2023-10-02 15:04:31.438 | DEBUG    | __main__:<module>:1 - <<<<<<<<<< INICIOU >>>>>>>>>>>
# 2023-10-02 15:04:41.479 | DEBUG    | __main__:<module>:2 - Department: DPT_25
# 2023-10-02 15:04:55.835 | DEBUG    | __main__:<module>:17 - [+] Start converting the text to one string
# 2023-10-02 15:05:05.548 | DEBUG    | __main__:<module>:19 - [+] End converting the text to one string
# 2023-10-02 15:05:05.549 | DEBUG    | __main__:<module>:21 - [+] Start removing duplicated words from the long string
# 2023-10-02 15:31:00.609 | INFO     | __main__:<module>:23 - unique_words_dts: 108843 

# 2023-10-02 15:31:00.610 | DEBUG    | __main__:<module>:24 - [+] End removing duplicated words from the long string
# 2023-10-02 15:31:00.656 | DEBUG    | __main__:<module>:37 - Department count samples: label                   4870
# department              4870
# entry_date              4870
# general_id              4870
# normative_identifier    4870
# publication_date        4870
# regulatory_authority    4870
# subject                 4870
# subject_length          4870
# subject_unique_words    4870
# subject_words           4870
# content                 4870
# text_length             4870
# text_unique_words       4870
# text_words              4870
# type                    4870
# unique_document_id      4870
# url                     4870
# new_content             4870
# dtype: int64
# 2023-10-02 15:31:00.660 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   819
# department              819
# entry_date              819
# general_id              819
# normative_identifier    819
# publication_date        819
# regulatory_authority    819
# subject                 819
# subject_length          819
# subject_unique_words    819
# subject_words           819
# content                 819
# text_length             819
# text_unique_words       819
# text_words              819
# type                    819
# unique_document_id      819
# url                     819
# new_content             819
# dtype: int64
# 2023-10-02 15:31:00.667 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   4051
# department              4051
# entry_date              4051
# general_id              4051
# normative_identifier    4051
# publication_date        4051
# regulatory_authority    4051
# subject                 4051
# subject_length          4051
# subject_unique_words    4051
# subject_words           4051
# content                 4051
# text_length             4051
# text_unique_words       4051
# text_words              4051
# type                    4051
# unique_document_id      4051
# url                     4051
# new_content             4051
# dtype: int64
# 2023-10-02 15:36:44.480 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-02 15:36:44.481 | INFO     | __main__:training_model:214 - Training loss: 0.4267598396069125
# 2023-10-02 15:37:02.341 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2224560117733384, [[-0.54227555  0.31132364]
 
# 2023-10-02 15:37:02.345 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7710843373493977
# 2023-10-02 15:37:02.347 | INFO     | __main__:training_model:220 - Validation loss: 0.2224560117733384
# 2023-10-02 15:37:02.347 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7710843373493977
# 2023-10-02 15:38:27.706 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-02 15:38:27.707 | INFO     | __main__:training_model:214 - Training loss: 0.19793415857773078
# 2023-10-02 15:38:45.634 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.1886841728396359, [[-1.3717551   0.97167164]
 
# 2023-10-02 15:38:45.637 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8148148148148148
# 2023-10-02 15:38:45.639 | INFO     | __main__:training_model:220 - Validation loss: 0.1886841728396359
# 2023-10-02 15:38:45.640 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8148148148148148
# 2023-10-02 15:40:10.836 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-02 15:40:10.838 | INFO     | __main__:training_model:214 - Training loss: 0.1556552997937328
# 2023-10-02 15:40:28.753 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.1746319200074862, [[-1.9820356  1.4770501]
 
# 2023-10-02 15:40:28.757 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.815873015873016
# 2023-10-02 15:40:28.759 | INFO     | __main__:training_model:220 - Validation loss: 0.1746319200074862
# 2023-10-02 15:40:28.760 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.815873015873016
# 2023-10-02 15:41:54.181 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-02 15:41:54.182 | INFO     | __main__:training_model:214 - Training loss: 0.13543951870187332
# 2023-10-02 15:42:12.109 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.15970400917447275, [[-2.138331   1.5381421]
 
# 2023-10-02 15:42:12.113 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8214285714285715
# 2023-10-02 15:42:12.114 | INFO     | __main__:training_model:220 - Validation loss: 0.15970400917447275
# 2023-10-02 15:42:12.115 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8214285714285715
# 2023-10-02 15:43:37.624 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-02 15:43:37.625 | INFO     | __main__:training_model:214 - Training loss: 0.12033461161742084
# 2023-10-02 15:43:55.568 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.16037319202922165, [[-2.1881218  1.5882444]
 
# 2023-10-02 15:43:55.572 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8184818481848185
# 2023-10-02 15:43:55.573 | INFO     | __main__:training_model:220 - Validation loss: 0.16037319202922165
# 2023-10-02 15:43:55.574 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8184818481848185
# 2023-10-02 15:44:15.138 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.16037319202922165, [[-2.1881218  1.5882444]
 
# 2023-10-02 15:44:15.140 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-02 15:44:15.142 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 1590/1620
# 2023-10-02 15:44:15.142 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9814814814814815

# 2023-10-02 15:44:15.143 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-02 15:44:15.144 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 248/328
# 2023-10-02 15:44:15.145 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.7560975609756098

# 2023-10-02 15:44:15.154 | INFO     | __main__:results:157 - {'0': {'precision': 0.9520958083832335, 'recall': 0.9814814814814815, 'f1-score': 0.9665653495440729, 'support': 1620}, '1': {'precision': 0.8920863309352518, 'recall': 0.7560975609756098, 'f1-score': 0.8184818481848185, 'support': 328}, 'accuracy': 0.9435318275154004, 'macro avg': {'precision': 0.9220910696592426, 'recall': 0.8687895212285457, 'f1-score': 0.8925235988644458, 'support': 1948}, 'weighted avg': {'precision': 0.9419915431866535, 'recall': 0.9435318275154004, 'f1-score': 0.9416313719024737, 'support': 1948}}
# 2023-10-02 15:44:15.156 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9520958083832335, 'recall': 0.9814814814814815, 'f1-score': 0.9665653495440729, 'support': 1620}, '1': {'precision': 0.8920863309352518, 'recall': 0.7560975609756098, 'f1-score': 0.8184818481848185, 'support': 328}, 'accuracy': 0.9435318275154004, 'macro avg': {'precision': 0.9220910696592426, 'recall': 0.8687895212285457, 'f1-score': 0.8925235988644458, 'support': 1948}, 'weighted avg': {'precision': 0.9419915431866535, 'recall': 0.9435318275154004, 'f1-score': 0.9416313719024737, 'support': 1948}}
# 2023-10-02 15:44:15.159 | DEBUG    | __main__:<module>:2 - Department: DPT_3
# 2023-10-02 15:44:35.649 | DEBUG    | __main__:<module>:17 - [+] Start converting the text to one string
# 2023-10-02 15:44:42.211 | DEBUG    | __main__:<module>:19 - [+] End converting the text to one string
# 2023-10-02 15:44:42.213 | DEBUG    | __main__:<module>:21 - [+] Start removing duplicated words from the long string
# 2023-10-02 16:05:54.226 | INFO     | __main__:<module>:23 - unique_words_dts: 65242 

# 2023-10-02 16:05:54.228 | DEBUG    | __main__:<module>:24 - [+] End removing duplicated words from the long string
# 2023-10-02 16:05:54.272 | DEBUG    | __main__:<module>:37 - Department count samples: label                   2478
# department              2478
# entry_date              2478
# general_id              2478
# normative_identifier    2478
# publication_date        2478
# regulatory_authority    2478
# subject                 2478
# subject_length          2478
# subject_unique_words    2478
# subject_words           2478
# content                 2478
# text_length             2478
# text_unique_words       2478
# text_words              2478
# type                    2478
# unique_document_id      2478
# url                     2478
# new_content             2478
# dtype: int64
# 2023-10-02 16:05:54.276 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   688
# department              688
# entry_date              688
# general_id              688
# normative_identifier    688
# publication_date        688
# regulatory_authority    688
# subject                 688
# subject_length          688
# subject_unique_words    688
# subject_words           688
# content                 688
# text_length             688
# text_unique_words       688
# text_words              688
# type                    688
# unique_document_id      688
# url                     688
# new_content             688
# dtype: int64
# 2023-10-02 16:05:54.281 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   1790
# department              1790
# entry_date              1790
# general_id              1790
# normative_identifier    1790
# publication_date        1790
# regulatory_authority    1790
# subject                 1790
# subject_length          1790
# subject_unique_words    1790
# subject_words           1790
# content                 1790
# text_length             1790
# text_unique_words       1790
# text_words              1790
# type                    1790
# unique_document_id      1790
# url                     1790
# new_content             1790
# dtype: int64
# 2023-10-02 16:11:53.349 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-02 16:11:53.351 | INFO     | __main__:training_model:214 - Training loss: 0.6198521784196297
# 2023-10-02 16:12:02.402 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5556126888841391, [[ 0.37988812 -0.6582369 ]
 
# 2023-10-02 16:12:02.405 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.0
# 2023-10-02 16:12:02.407 | INFO     | __main__:training_model:220 - Validation loss: 0.5556126888841391
# 2023-10-02 16:12:02.408 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.0
# 2023-10-02 16:12:46.524 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-02 16:12:46.526 | INFO     | __main__:training_model:214 - Training loss: 0.5390354494253794
# 2023-10-02 16:12:55.641 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4737525638192892, [[ 0.23549332 -0.5993007 ]
 
# 2023-10-02 16:12:55.645 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5877862595419847
# 2023-10-02 16:12:55.647 | INFO     | __main__:training_model:220 - Validation loss: 0.4737525638192892
# 2023-10-02 16:12:55.648 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5877862595419847
# 2023-10-02 16:13:40.023 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-02 16:13:40.024 | INFO     | __main__:training_model:214 - Training loss: 0.4344751586516698
# 2023-10-02 16:13:49.159 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.36514051572885364, [[ 0.00477344 -0.28470698]
 
# 2023-10-02 16:13:49.162 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6652631578947368
# 2023-10-02 16:13:49.164 | INFO     | __main__:training_model:220 - Validation loss: 0.36514051572885364
# 2023-10-02 16:13:49.165 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6652631578947368
# 2023-10-02 16:14:32.978 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-02 16:14:32.979 | INFO     | __main__:training_model:214 - Training loss: 0.3637825104718407
# 2023-10-02 16:14:42.118 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.36758520803414285, [[-0.23257986 -0.01623377]
 
# 2023-10-02 16:14:42.121 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6824817518248176
# 2023-10-02 16:14:42.123 | INFO     | __main__:training_model:220 - Validation loss: 0.36758520803414285
# 2023-10-02 16:14:42.124 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6824817518248176
# 2023-10-02 16:15:26.576 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-02 16:15:26.577 | INFO     | __main__:training_model:214 - Training loss: 0.3358484758064151
# 2023-10-02 16:15:35.725 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3674958246992901, [[-0.13519186 -0.10498893]
 
# 2023-10-02 16:15:35.728 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6931818181818182
# 2023-10-02 16:15:35.730 | INFO     | __main__:training_model:220 - Validation loss: 0.3674958246992901
# 2023-10-02 16:15:35.731 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6931818181818182
# 2023-10-02 16:15:46.242 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3674958246992901, [[-0.13519186 -0.10498893]
 
# 2023-10-02 16:15:46.243 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-02 16:15:46.244 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 647/717
# 2023-10-02 16:15:46.245 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.902370990237099

# 2023-10-02 16:15:46.246 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-02 16:15:46.247 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 183/275
# 2023-10-02 16:15:46.248 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6654545454545454

# 2023-10-02 16:15:46.256 | INFO     | __main__:results:157 - {'0': {'precision': 0.8755074424898511, 'recall': 0.902370990237099, 'f1-score': 0.8887362637362638, 'support': 717}, '1': {'precision': 0.7233201581027668, 'recall': 0.6654545454545454, 'f1-score': 0.6931818181818182, 'support': 275}, 'accuracy': 0.8366935483870968, 'macro avg': {'precision': 0.7994138002963089, 'recall': 0.7839127678458222, 'f1-score': 0.790959040959041, 'support': 992}, 'weighted avg': {'precision': 0.8333184271607703, 'recall': 0.8366935483870968, 'f1-score': 0.8345251019142148, 'support': 992}}
# 2023-10-02 16:15:46.258 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8755074424898511, 'recall': 0.902370990237099, 'f1-score': 0.8887362637362638, 'support': 717}, '1': {'precision': 0.7233201581027668, 'recall': 0.6654545454545454, 'f1-score': 0.6931818181818182, 'support': 275}, 'accuracy': 0.8366935483870968, 'macro avg': {'precision': 0.7994138002963089, 'recall': 0.7839127678458222, 'f1-score': 0.790959040959041, 'support': 992}, 'weighted avg': {'precision': 0.8333184271607703, 'recall': 0.8366935483870968, 'f1-score': 0.8345251019142148, 'support': 992}}
# 2023-10-02 16:15:46.265 | DEBUG    | __main__:<module>:2 - Department: DPT_2
# 2023-10-02 16:16:11.201 | DEBUG    | __main__:<module>:17 - [+] Start converting the text to one string
# 2023-10-02 16:16:25.612 | DEBUG    | __main__:<module>:19 - [+] End converting the text to one string
# 2023-10-02 16:16:25.613 | DEBUG    | __main__:<module>:21 - [+] Start removing duplicated words from the long string
# 2023-10-02 16:48:22.504 | INFO     | __main__:<module>:23 - unique_words_dts: 83183 

# 2023-10-02 16:48:22.506 | DEBUG    | __main__:<module>:24 - [+] End removing duplicated words from the long string
# 2023-10-02 16:48:22.554 | DEBUG    | __main__:<module>:37 - Department count samples: label                   5917
# department              5917
# entry_date              5917
# general_id              5917
# normative_identifier    5917
# publication_date        5917
# regulatory_authority    5917
# subject                 5917
# subject_length          5917
# subject_unique_words    5917
# subject_words           5917
# content                 5917
# text_length             5917
# text_unique_words       5917
# text_words              5917
# type                    5917
# unique_document_id      5917
# url                     5917
# new_content             5917
# dtype: int64
# 2023-10-02 16:48:22.559 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   683
# department              683
# entry_date              683
# general_id              683
# normative_identifier    683
# publication_date        683
# regulatory_authority    683
# subject                 683
# subject_length          683
# subject_unique_words    683
# subject_words           683
# content                 683
# text_length             683
# text_unique_words       683
# text_words              683
# type                    683
# unique_document_id      683
# url                     683
# new_content             683
# dtype: int64
# 2023-10-02 16:48:22.565 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   5234
# department              5234
# entry_date              5234
# general_id              5234
# normative_identifier    5234
# publication_date        5234
# regulatory_authority    5234
# subject                 5234
# subject_length          5234
# subject_unique_words    5234
# subject_words           5234
# content                 5234
# text_length             5234
# text_unique_words       5234
# text_words              5234
# type                    5234
# unique_document_id      5234
# url                     5234
# new_content             5234
# dtype: int64
# 2023-10-02 16:56:35.864 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-02 16:56:35.865 | INFO     | __main__:training_model:214 - Training loss: 0.3832617528412653
# 2023-10-02 16:56:57.611 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2663505965142281, [[-0.0471767   0.49994525]
 
# 2023-10-02 16:56:57.615 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5477941176470589
# 2023-10-02 16:56:57.616 | INFO     | __main__:training_model:220 - Validation loss: 0.2663505965142281
# 2023-10-02 16:56:57.617 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5477941176470589
# 2023-10-02 16:58:40.465 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-02 16:58:40.466 | INFO     | __main__:training_model:214 - Training loss: 0.22798783856889476
# 2023-10-02 16:59:02.261 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.23303805693209945, [[-0.34020284  0.78242165]
 
# 2023-10-02 16:59:02.265 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6069651741293532
# 2023-10-02 16:59:02.266 | INFO     | __main__:training_model:220 - Validation loss: 0.23303805693209945
# 2023-10-02 16:59:02.267 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6069651741293532
# 2023-10-02 17:00:44.884 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-02 17:00:44.885 | INFO     | __main__:training_model:214 - Training loss: 0.18721457761914834
# 2023-10-02 17:01:06.694 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2327398223161407, [[-0.42202505  0.82722336]
 
# 2023-10-02 17:01:06.697 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6141215106732348
# 2023-10-02 17:01:06.698 | INFO     | __main__:training_model:220 - Validation loss: 0.2327398223161407
# 2023-10-02 17:01:06.699 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6141215106732348
# 2023-10-02 17:02:49.693 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-02 17:02:49.694 | INFO     | __main__:training_model:214 - Training loss: 0.15731601122280825
# 2023-10-02 17:03:11.516 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.24609348296441816, [[-0.6031799   1.0470887 ]
 
# 2023-10-02 17:03:11.520 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6057838660578386
# 2023-10-02 17:03:11.521 | INFO     | __main__:training_model:220 - Validation loss: 0.24609348296441816
# 2023-10-02 17:03:11.522 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6057838660578386
# 2023-10-02 17:04:54.690 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-02 17:04:54.692 | INFO     | __main__:training_model:214 - Training loss: 0.13931606993403123
# 2023-10-02 17:05:16.496 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.23923234350060102, [[-0.63756263  1.049917  ]
 
# 2023-10-02 17:05:16.499 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6033333333333334
# 2023-10-02 17:05:16.501 | INFO     | __main__:training_model:220 - Validation loss: 0.23923234350060102
# 2023-10-02 17:05:16.501 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6033333333333334
# 2023-10-02 17:05:39.677 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.23923234350060102, [[-0.63756263  1.049917  ]
 
# 2023-10-02 17:05:39.679 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-02 17:05:39.680 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 1948/2094
# 2023-10-02 17:05:39.681 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9302769818529131

# 2023-10-02 17:05:39.682 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-02 17:05:39.682 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 181/273
# 2023-10-02 17:05:39.683 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.663003663003663

# 2023-10-02 17:05:39.693 | INFO     | __main__:results:157 - {'0': {'precision': 0.9549019607843138, 'recall': 0.9302769818529131, 'f1-score': 0.9424286405418482, 'support': 2094}, '1': {'precision': 0.5535168195718655, 'recall': 0.663003663003663, 'f1-score': 0.6033333333333334, 'support': 273}, 'accuracy': 0.8994507815800592, 'macro avg': {'precision': 0.7542093901780896, 'recall': 0.7966403224282881, 'f1-score': 0.7728809869375908, 'support': 2367}, 'weighted avg': {'precision': 0.908607857044982, 'recall': 0.8994507815800592, 'f1-score': 0.9033187888866202, 'support': 2367}}
# 2023-10-02 17:05:39.695 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9549019607843138, 'recall': 0.9302769818529131, 'f1-score': 0.9424286405418482, 'support': 2094}, '1': {'precision': 0.5535168195718655, 'recall': 0.663003663003663, 'f1-score': 0.6033333333333334, 'support': 273}, 'accuracy': 0.8994507815800592, 'macro avg': {'precision': 0.7542093901780896, 'recall': 0.7966403224282881, 'f1-score': 0.7728809869375908, 'support': 2367}, 'weighted avg': {'precision': 0.908607857044982, 'recall': 0.8994507815800592, 'f1-score': 0.9033187888866202, 'support': 2367}}
# 2023-10-02 17:05:39.702 | DEBUG    | __main__:<module>:2 - Department: DPT_9
# 2023-10-02 17:05:53.821 | DEBUG    | __main__:<module>:17 - [+] Start converting the text to one string
# 2023-10-02 17:06:02.487 | DEBUG    | __main__:<module>:19 - [+] End converting the text to one string
# 2023-10-02 17:06:02.489 | DEBUG    | __main__:<module>:21 - [+] Start removing duplicated words from the long string
# 2023-10-02 17:25:39.044 | INFO     | __main__:<module>:23 - unique_words_dts: 57951 

# 2023-10-02 17:25:39.045 | DEBUG    | __main__:<module>:24 - [+] End removing duplicated words from the long string
# 2023-10-02 17:25:39.092 | DEBUG    | __main__:<module>:37 - Department count samples: label                   4219
# department              4219
# entry_date              4219
# general_id              4219
# normative_identifier    4219
# publication_date        4219
# regulatory_authority    4219
# subject                 4219
# subject_length          4219
# subject_unique_words    4219
# subject_words           4219
# content                 4219
# text_length             4219
# text_unique_words       4219
# text_words              4219
# type                    4219
# unique_document_id      4219
# url                     4219
# new_content             4219
# dtype: int64
# 2023-10-02 17:25:39.096 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   600
# department              600
# entry_date              600
# general_id              600
# normative_identifier    600
# publication_date        600
# regulatory_authority    600
# subject                 600
# subject_length          600
# subject_unique_words    600
# subject_words           600
# content                 600
# text_length             600
# text_unique_words       600
# text_words              600
# type                    600
# unique_document_id      600
# url                     600
# new_content             600
# dtype: int64
# 2023-10-02 17:25:39.101 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   3619
# department              3619
# entry_date              3619
# general_id              3619
# normative_identifier    3619
# publication_date        3619
# regulatory_authority    3619
# subject                 3619
# subject_length          3619
# subject_unique_words    3619
# subject_words           3619
# content                 3619
# text_length             3619
# text_unique_words       3619
# text_words              3619
# type                    3619
# unique_document_id      3619
# url                     3619
# new_content             3619
# dtype: int64
# 2023-10-02 17:30:31.537 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-02 17:30:31.539 | INFO     | __main__:training_model:214 - Training loss: 0.46285779665156107
# 2023-10-02 17:30:47.018 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.37282448424534365, [[ 0.7857302  -0.71896076]
 
# 2023-10-02 17:30:47.022 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.0
# 2023-10-02 17:30:47.023 | INFO     | __main__:training_model:220 - Validation loss: 0.37282448424534365
# 2023-10-02 17:30:47.024 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.0
# 2023-10-02 17:32:00.707 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-02 17:32:00.708 | INFO     | __main__:training_model:214 - Training loss: 0.31453635171055794
# 2023-10-02 17:32:16.246 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.26960634250532495, [[ 0.05765912  0.44334182]
 
# 2023-10-02 17:32:16.249 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.4538653366583541
# 2023-10-02 17:32:16.251 | INFO     | __main__:training_model:220 - Validation loss: 0.26960634250532495
# 2023-10-02 17:32:16.251 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.4538653366583541
# 2023-10-02 17:33:30.273 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-02 17:33:30.275 | INFO     | __main__:training_model:214 - Training loss: 0.23422339985646853
# 2023-10-02 17:33:45.828 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.25623450899835337, [[-0.2233289   0.7648068 ]
 
# 2023-10-02 17:33:45.832 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5751633986928105
# 2023-10-02 17:33:45.833 | INFO     | __main__:training_model:220 - Validation loss: 0.25623450899835337
# 2023-10-02 17:33:45.834 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5751633986928105
# 2023-10-02 17:34:59.749 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-02 17:34:59.750 | INFO     | __main__:training_model:214 - Training loss: 0.19583767620710338
# 2023-10-02 17:35:15.311 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.27331001153215767, [[-0.6686388   1.2601074 ]
 
# 2023-10-02 17:35:15.315 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.603290676416819
# 2023-10-02 17:35:15.316 | INFO     | __main__:training_model:220 - Validation loss: 0.27331001153215767
# 2023-10-02 17:35:15.317 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.603290676416819
# 2023-10-02 17:36:29.387 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-02 17:36:29.388 | INFO     | __main__:training_model:214 - Training loss: 0.1702191352208213
# 2023-10-02 17:36:44.954 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2694872633638707, [[-0.6817699   1.2601359 ]
 
# 2023-10-02 17:36:44.958 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6015325670498084
# 2023-10-02 17:36:44.960 | INFO     | __main__:training_model:220 - Validation loss: 0.2694872633638707
# 2023-10-02 17:36:44.961 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6015325670498084
# 2023-10-02 17:37:01.956 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2694872633638707, [[-0.6817699   1.2601359 ]
 
# 2023-10-02 17:37:01.958 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-02 17:37:01.959 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 1323/1448
# 2023-10-02 17:37:01.960 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9136740331491713

# 2023-10-02 17:37:01.961 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-02 17:37:01.962 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 157/240
# 2023-10-02 17:37:01.963 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6541666666666667

# 2023-10-02 17:37:01.972 | INFO     | __main__:results:157 - {'0': {'precision': 0.9409672830725462, 'recall': 0.9136740331491713, 'f1-score': 0.9271198318149965, 'support': 1448}, '1': {'precision': 0.5567375886524822, 'recall': 0.6541666666666667, 'f1-score': 0.6015325670498084, 'support': 240}, 'accuracy': 0.8767772511848341, 'macro avg': {'precision': 0.7488524358625142, 'recall': 0.783920349907919, 'f1-score': 0.7643261994324024, 'support': 1688}, 'weighted avg': {'precision': 0.8863374687000253, 'recall': 0.8767772511848341, 'f1-score': 0.8808278036493299, 'support': 1688}}
# 2023-10-02 17:37:01.974 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9409672830725462, 'recall': 0.9136740331491713, 'f1-score': 0.9271198318149965, 'support': 1448}, '1': {'precision': 0.5567375886524822, 'recall': 0.6541666666666667, 'f1-score': 0.6015325670498084, 'support': 240}, 'accuracy': 0.8767772511848341, 'macro avg': {'precision': 0.7488524358625142, 'recall': 0.783920349907919, 'f1-score': 0.7643261994324024, 'support': 1688}, 'weighted avg': {'precision': 0.8863374687000253, 'recall': 0.8767772511848341, 'f1-score': 0.8808278036493299, 'support': 1688}}
# 2023-10-02 17:37:01.981 | DEBUG    | __main__:<module>:2 - Department: DPT_5
# 2023-10-02 17:37:11.709 | DEBUG    | __main__:<module>:17 - [+] Start converting the text to one string
# 2023-10-02 17:37:16.372 | DEBUG    | __main__:<module>:19 - [+] End converting the text to one string
# 2023-10-02 17:37:16.374 | DEBUG    | __main__:<module>:21 - [+] Start removing duplicated words from the long string
# 2023-10-02 17:50:31.780 | INFO     | __main__:<module>:23 - unique_words_dts: 69459 

# 2023-10-02 17:50:31.781 | DEBUG    | __main__:<module>:24 - [+] End removing duplicated words from the long string
# 2023-10-02 17:50:31.829 | DEBUG    | __main__:<module>:37 - Department count samples: label                   3128
# department              3128
# entry_date              3128
# general_id              3128
# normative_identifier    3128
# publication_date        3128
# regulatory_authority    3128
# subject                 3128
# subject_length          3128
# subject_unique_words    3128
# subject_words           3128
# content                 3128
# text_length             3128
# text_unique_words       3128
# text_words              3128
# type                    3128
# unique_document_id      3128
# url                     3128
# new_content             3128
# dtype: int64
# 2023-10-02 17:50:31.834 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   534
# department              534
# entry_date              534
# general_id              534
# normative_identifier    534
# publication_date        534
# regulatory_authority    534
# subject                 534
# subject_length          534
# subject_unique_words    534
# subject_words           534
# content                 534
# text_length             534
# text_unique_words       534
# text_words              534
# type                    534
# unique_document_id      534
# url                     534
# new_content             534
# dtype: int64
# 2023-10-02 17:50:31.839 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   2594
# department              2594
# entry_date              2594
# general_id              2594
# normative_identifier    2594
# publication_date        2594
# regulatory_authority    2594
# subject                 2594
# subject_length          2594
# subject_unique_words    2594
# subject_words           2594
# content                 2594
# text_length             2594
# text_unique_words       2594
# text_words              2594
# type                    2594
# unique_document_id      2594
# url                     2594
# new_content             2594
# dtype: int64
# 2023-10-02 17:54:08.869 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-02 17:54:08.870 | INFO     | __main__:training_model:214 - Training loss: 0.520693270397968
# 2023-10-02 17:54:20.319 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.42932114259498877, [[ 0.38391146 -0.8829357 ]
 
# 2023-10-02 17:54:20.323 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.0
# 2023-10-02 17:54:20.324 | INFO     | __main__:training_model:220 - Validation loss: 0.42932114259498877
# 2023-10-02 17:54:20.325 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.0
# 2023-10-02 17:55:15.615 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-02 17:55:15.617 | INFO     | __main__:training_model:214 - Training loss: 0.39496537701028295
# 2023-10-02 17:55:27.107 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3493135978535908, [[ 0.3150764  -0.5949878 ]
 
# 2023-10-02 17:55:27.110 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.39067055393586003
# 2023-10-02 17:55:27.112 | INFO     | __main__:training_model:220 - Validation loss: 0.3493135978535908
# 2023-10-02 17:55:27.113 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.39067055393586003
# 2023-10-02 17:56:22.314 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-02 17:56:22.315 | INFO     | __main__:training_model:214 - Training loss: 0.3381454868883383
# 2023-10-02 17:56:33.839 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3380048139611395, [[ 0.310631   -0.5467653 ]
 
# 2023-10-02 17:56:33.843 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.40978593272171254
# 2023-10-02 17:56:33.844 | INFO     | __main__:training_model:220 - Validation loss: 0.3380048139611395
# 2023-10-02 17:56:33.845 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.40978593272171254
# 2023-10-02 17:57:29.209 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-02 17:57:29.211 | INFO     | __main__:training_model:214 - Training loss: 0.30098520805601214
# 2023-10-02 17:57:40.736 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3367139538068597, [[ 0.11652394 -0.4767127 ]
 
# 2023-10-02 17:57:40.739 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5144356955380578
# 2023-10-02 17:57:40.740 | INFO     | __main__:training_model:220 - Validation loss: 0.3367139538068597
# 2023-10-02 17:57:40.741 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5144356955380578
# 2023-10-02 17:58:36.201 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-02 17:58:36.202 | INFO     | __main__:training_model:214 - Training loss: 0.2757293301039055
# 2023-10-02 17:58:47.736 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3378323729990459, [[ 0.04382604 -0.27015802]
 
# 2023-10-02 17:58:47.739 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5223880597014925
# 2023-10-02 17:58:47.741 | INFO     | __main__:training_model:220 - Validation loss: 0.3378323729990459
# 2023-10-02 17:58:47.741 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5223880597014925
# 2023-10-02 17:59:00.720 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3378323729990459, [[ 0.04382604 -0.27015802]
 
# 2023-10-02 17:59:00.722 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-02 17:59:00.723 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 955/1038
# 2023-10-02 17:59:00.723 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9200385356454721

# 2023-10-02 17:59:00.724 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-02 17:59:00.724 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 105/214
# 2023-10-02 17:59:00.725 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.49065420560747663

# 2023-10-02 17:59:00.733 | INFO     | __main__:results:157 - {'0': {'precision': 0.8975563909774437, 'recall': 0.9200385356454721, 'f1-score': 0.9086584205518554, 'support': 1038}, '1': {'precision': 0.5585106382978723, 'recall': 0.49065420560747663, 'f1-score': 0.5223880597014925, 'support': 214}, 'accuracy': 0.8466453674121406, 'macro avg': {'precision': 0.728033514637658, 'recall': 0.7053463706264744, 'f1-score': 0.715523240126674, 'support': 1252}, 'weighted avg': {'precision': 0.839604481174386, 'recall': 0.8466453674121406, 'f1-score': 0.8426345729304674, 'support': 1252}}
# 2023-10-02 17:59:00.735 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8975563909774437, 'recall': 0.9200385356454721, 'f1-score': 0.9086584205518554, 'support': 1038}, '1': {'precision': 0.5585106382978723, 'recall': 0.49065420560747663, 'f1-score': 0.5223880597014925, 'support': 214}, 'accuracy': 0.8466453674121406, 'macro avg': {'precision': 0.728033514637658, 'recall': 0.7053463706264744, 'f1-score': 0.715523240126674, 'support': 1252}, 'weighted avg': {'precision': 0.839604481174386, 'recall': 0.8466453674121406, 'f1-score': 0.8426345729304674, 'support': 1252}}
# 2023-10-02 17:59:00.738 | DEBUG    | __main__:<module>:2 - Department: DPT_16
# 2023-10-02 17:59:10.595 | DEBUG    | __main__:<module>:17 - [+] Start converting the text to one string
# 2023-10-02 17:59:14.746 | DEBUG    | __main__:<module>:19 - [+] End converting the text to one string
# 2023-10-02 17:59:14.747 | DEBUG    | __main__:<module>:21 - [+] Start removing duplicated words from the long string
# 2023-10-02 18:12:15.963 | INFO     | __main__:<module>:23 - unique_words_dts: 58224 

# 2023-10-02 18:12:15.964 | DEBUG    | __main__:<module>:24 - [+] End removing duplicated words from the long string
# 2023-10-02 18:12:16.010 | DEBUG    | __main__:<module>:37 - Department count samples: label                   1997
# department              1997
# entry_date              1997
# general_id              1997
# normative_identifier    1997
# publication_date        1997
# regulatory_authority    1997
# subject                 1997
# subject_length          1997
# subject_unique_words    1997
# subject_words           1997
# content                 1997
# text_length             1997
# text_unique_words       1997
# text_words              1997
# type                    1997
# unique_document_id      1997
# url                     1997
# new_content             1997
# dtype: int64
# 2023-10-02 18:12:16.015 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   439
# department              439
# entry_date              439
# general_id              439
# normative_identifier    439
# publication_date        439
# regulatory_authority    439
# subject                 439
# subject_length          439
# subject_unique_words    439
# subject_words           439
# content                 439
# text_length             439
# text_unique_words       439
# text_words              439
# type                    439
# unique_document_id      439
# url                     439
# new_content             439
# dtype: int64
# 2023-10-02 18:12:16.019 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   1558
# department              1558
# entry_date              1558
# general_id              1558
# normative_identifier    1558
# publication_date        1558
# regulatory_authority    1558
# subject                 1558
# subject_length          1558
# subject_unique_words    1558
# subject_words           1558
# content                 1558
# text_length             1558
# text_unique_words       1558
# text_words              1558
# type                    1558
# unique_document_id      1558
# url                     1558
# new_content             1558
# dtype: int64
# 2023-10-02 18:15:23.354 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-02 18:15:23.355 | INFO     | __main__:training_model:214 - Training loss: 0.5822781721750895
# 2023-10-02 18:15:30.614 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5093899575563577, [[ 0.6021688  -0.7266697 ]
 
# 2023-10-02 18:15:30.617 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.0
# 2023-10-02 18:15:30.619 | INFO     | __main__:training_model:220 - Validation loss: 0.5093899575563577
# 2023-10-02 18:15:30.620 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.0
# 2023-10-02 18:16:06.379 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-02 18:16:06.380 | INFO     | __main__:training_model:214 - Training loss: 0.47445766360331804
# 2023-10-02 18:16:13.710 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.43185004792534387, [[ 0.6931313  -0.7391861 ]
 
# 2023-10-02 18:16:13.714 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.3318385650224216
# 2023-10-02 18:16:13.715 | INFO     | __main__:training_model:220 - Validation loss: 0.43185004792534387
# 2023-10-02 18:16:13.716 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.3318385650224216
# 2023-10-02 18:16:48.989 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-02 18:16:48.990 | INFO     | __main__:training_model:214 - Training loss: 0.40158439446718264
# 2023-10-02 18:16:56.342 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.37930412217974663, [[-0.00198019  0.35735616]
 
# 2023-10-02 18:16:56.345 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.49056603773584906
# 2023-10-02 18:16:56.347 | INFO     | __main__:training_model:220 - Validation loss: 0.37930412217974663
# 2023-10-02 18:16:56.348 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.49056603773584906
# 2023-10-02 18:17:32.398 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-02 18:17:32.399 | INFO     | __main__:training_model:214 - Training loss: 0.3467003959111678
# 2023-10-02 18:17:39.751 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3837421638174699, [[-0.08544118  0.36705264]
 
# 2023-10-02 18:17:39.755 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.4923076923076923
# 2023-10-02 18:17:39.756 | INFO     | __main__:training_model:220 - Validation loss: 0.3837421638174699
# 2023-10-02 18:17:39.757 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.4923076923076923
# 2023-10-02 18:18:15.231 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-02 18:18:15.234 | INFO     | __main__:training_model:214 - Training loss: 0.31403095179643387
# 2023-10-02 18:18:22.597 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.36652974483485407, [[-0.34888595  0.7059946 ]
 
# 2023-10-02 18:18:22.600 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.56047197640118
# 2023-10-02 18:18:22.602 | INFO     | __main__:training_model:220 - Validation loss: 0.36652974483485407
# 2023-10-02 18:18:22.602 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.56047197640118
# 2023-10-02 18:18:31.385 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.36652974483485407, [[-0.34888595  0.7059946 ]
 
# 2023-10-02 18:18:31.387 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-02 18:18:31.388 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 555/623
# 2023-10-02 18:18:31.389 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8908507223113965

# 2023-10-02 18:18:31.389 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-02 18:18:31.390 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 95/176
# 2023-10-02 18:18:31.390 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.5397727272727273

# 2023-10-02 18:18:31.398 | INFO     | __main__:results:157 - {'0': {'precision': 0.8726415094339622, 'recall': 0.8908507223113965, 'f1-score': 0.8816521048451151, 'support': 623}, '1': {'precision': 0.5828220858895705, 'recall': 0.5397727272727273, 'f1-score': 0.56047197640118, 'support': 176}, 'accuracy': 0.8135168961201502, 'macro avg': {'precision': 0.7277317976617663, 'recall': 0.715311724792062, 'f1-score': 0.7210620406231476, 'support': 799}, 'weighted avg': {'precision': 0.8088014361626068, 'recall': 0.8135168961201502, 'f1-score': 0.8109040415082784, 'support': 799}}
# 2023-10-02 18:18:31.400 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8726415094339622, 'recall': 0.8908507223113965, 'f1-score': 0.8816521048451151, 'support': 623}, '1': {'precision': 0.5828220858895705, 'recall': 0.5397727272727273, 'f1-score': 0.56047197640118, 'support': 176}, 'accuracy': 0.8135168961201502, 'macro avg': {'precision': 0.7277317976617663, 'recall': 0.715311724792062, 'f1-score': 0.7210620406231476, 'support': 799}, 'weighted avg': {'precision': 0.8088014361626068, 'recall': 0.8135168961201502, 'f1-score': 0.8109040415082784, 'support': 799}}
